# Library

In [3]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import pyexasol

In [4]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [5]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [6]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [16]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
wz = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSZ')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# 로드

## 로드-등록정보

In [65]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소여부',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print(carr.shape)
print('data load : STD_CEG_CAR_MIG')

(31131025, 15)
data load : STD_CEG_CAR_MIG


In [66]:
print(carr['배출가스등급'].unique())
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
print(carr['배출가스등급'].unique())

print(carr['차종'].unique())
# 4.0s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
print(carr['차종'].unique())

print(carr['용도'].unique())
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
print(carr['용도'].unique())

['A0503' 'A0502' 'A0505' 'A0501' 'A0504' nan]
['3' '2' '5' '1' '4' nan]
['A31P' 'A31T' 'A31V' 'A31S']
['승용' '화물' '승합' '특수']
['A08P' 'A08B' 'A08O']
['개인용' '영업용' '관용']


## 로드-제원정보

In [67]:
# 4.2s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.shape)
print('data load : STD_CEG_CAR_SRC_MIG')

(574467, 12)
data load : STD_CEG_CAR_SRC_MIG


In [68]:
print(srcr['연료'].unique())
# 7.9s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
print(srcr['연료'].unique())

print(srcr['차종유형'].unique())
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
print(srcr['차종유형'].unique())

['A90GS' 'A94OT' 'A91KS' 'A93EV' 'A91DS' nan 'A92LP' 'A90GH' 'A92CN'
 'A90AC' 'A93SH' 'A91DH' 'A92LH' 'A93HD' 'A92LN' 'A92CH']
['휘발유' '기타연료' '등유' '전기' '경유' nan 'LPG(액화석유가스)' '휘발유 하이브리드' 'CNG(압축천연가스)'
 '알코올' '태양열' '경유 하이브리드' 'LPG 하이브리드' '수소' 'LNG(액화천연가스)' 'CNG 하이브리드']
['A30L' 'A30M' 'A30S' 'A30C' nan]
['대형' '중형' '소형' '경형' nan]


## 법정동코드

In [69]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SGG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SGG_CD':'시군구코드', 
}
coder = code.rename(columns=code_ch_col)

print(coder.columns)
print(coder.shape)
print('data load : STD_BJCD_INFO')

Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')
(47916, 5)
data load : STD_BJCD_INFO


## 로드-LST

### from DB

In [70]:
# 7m 4.8s
# cs_raw = wz.export_to_pandas("SELECT * FROM VSYSZ.STD_CEG_GRD_LST WHERE CLS_YMD = (SELECT MAX(CLS_YMD) FROM VSYSZ.STD_CEG_GRD_LST);") # 최신 12월 31일자
cs_raw = wz.export_to_pandas("SELECT * FROM VSYSZ.STD_CEG_GRD_LST WHERE CLS_YMD = '20231130';") # 11월 30일자
cs_raw_dict = {
    'CLS_YMD':'원천데이터생성일자',

    'VIN':'차대번호', 
    'MANP_MNG_NO':'제원관리번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'YRIDNW':'연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    'VHCTY_TY_CD2':'차종유형', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',

    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',

    'VHCTY_CL_CD':'차종분류',
    'CRYNG_WGHT':'적재중량',

    'STDG_CTPV_NM':'시도명',
    'STDG_SGG_NM':'시군구명',
    'LEM_KND_CD':'저공해조치종류코드',
    'STRCT_CHG_YN':'구조변경여부',
    'FUEL_G_NM':'연료_G_명',
    'TKCAR_PSCP':'승차정원',
    'OVRLH':'전장',
    'OVRWD':'전폭',
    'OVRHG':'전고',
    'FBCTN_NTN_NM':'제작국명',
    'BOHUM_YN':'보훈여부',
}

cs = cs_raw.rename(columns=cs_raw_dict)

print(cs['원천데이터생성일자'].unique()[0])
print(cs.columns)
print(cs.shape)
print('data load : STD_CEG_GRD_LST')

20231130
Index(['원천데이터생성일자', '차량관리번호', '자동차등록번호', '법정동코드', '시도명', '시군구명', '소유자구분', '차종',
       '용도', '차대번호', '최초등록일자', '연식', '제작일자', '검사유효일', '배출가스인증번호', '배출가스등급',
       '저공해조치종류코드', '구조변경여부', '차량말소여부', '제원관리번호', '차명', '차종분류', '차종유형',
       '자동차형식', '제작사명', '연료', '연료_G_명', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차정원', '전장', '전폭', '전고', '제작국명', '보훈여부'],
      dtype='object')
(31367557, 38)
data load : STD_CEG_GRD_LST


In [142]:
cs.drop_duplicates().shape

(31367557, 38)

#### [출력] STD_CEG_GRD_LST

In [71]:
today_date_dtl= datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105_PM_120814'

In [72]:
# # 3m 16.5s
# data_date = cs['원천데이터생성일자'].unique()[0]
# cs.to_csv(os.path.join(pub_fold, f'STD_CEG_GRD_LST_{data_date}_{today_date_dtl}.csv'), index=False, sep='|')

### from csv

In [73]:
# # 3m 15.1s
# cs_raw_dict = {
#     'CLS_YMD':'원천데이터생성일자',
    
#     'VIN':'차대번호', 
#     'MANP_MNG_NO':'제원관리번호', 
#     'VHCL_ERSR_YN':'차량말소여부',
#     'FRST_REG_YMD':'최초등록일자',
#     'YRIDNW':'연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'BSPL_STDG_CD':'법정동코드', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'FUEL_CD':'연료',
#     'MNFCTR_NM':'제작사명', 
#     'VHCTY_TY_CD2':'차종유형', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'TOTL_WGHT':'총중량',
#     'DSPLVL':'배기량', 
#     'EGIN_OTPT':'엔진출력',

#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_FBCTN_YMD':'제작일자', 
#     'VHCL_MNG_NO':'차량관리번호', 
#     'VHRNO':'자동차등록번호',
#     'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     'VHCL_OWNR_CL_CD':'소유자구분',

#     'VHCTY_CL_CD':'차종분류',
#     'CRYNG_WGHT':'적재중량',

#     'STDG_CTPV_NM':'시도명',
#     'STDG_SGG_NM':'시군구명',
#     'LEM_KND_CD':'저공해조치종류코드',
#     'STRCT_CHG_YN':'구조변경여부',
#     'FUEL_G_NM':'연료_G_명',
#     'TKCAR_PSCP':'승차정원',
#     'OVRLH':'전장',
#     'OVRWD':'전폭',
#     'OVRHG':'전고',
#     'FBCTN_NTN_NM':'제작국명',
#     'BOHUM_YN':'보훈여부',
# }

# name = 'STD_CEG_GRD_LST_20231231_20240105_AM_095035'
# file_name = f'{name}.csv'
# cs_raw = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False, usecols=cs_raw_dict.keys())
# cs = cs_raw.rename(columns=cs_raw_dict)
# print(cs.columns)
# print(cs.shape)
# print('data load : STD_CEG_GRD_LST')

In [74]:
# 31.8s
# 코드 변환
## 배출가스등급 코드 변환
print(cs['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
print(cs['배출가스등급'].unique())
print()

## 차종 코드 변환
print(cs['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
print(cs['차종'].unique())
print()

## 용도 코드 변환
print(cs['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
cs['용도'].unique()
print()

## 소유자구분 코드 변환
print(cs['소유자구분'].unique())
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
print(cs['소유자구분'].unique())
print()

## 연료 코드 변환
print(cs['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
print(cs['연료'].unique())
print()

## 차종유형 코드 변환
print(cs['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
print(cs['차종유형'].unique())

['A0504' 'A0503' 'A0505' 'A0502' 'A0501' 'X']
['4' '3' '5' '2' '1' 'X']

['A31P' 'A31T' 'A31V' 'A31S']
['승용' '화물' '승합' '특수']

['A08P' 'A08B' 'A08O']

['A27R' 'A27L' 'A27B' 'A27F' 'A27O']
['주민' '법인' '사업자' '외국인' '기타']

['A90GS' 'A92LP' 'A91DS' 'A90GH' nan 'A94OT' 'A92CN' 'A93EV' 'A92LH'
 'A91DH' 'A90AC' 'A91KS' 'A93HD' 'A93SH' 'A92LN' 'A92CH']
['휘발유' 'LPG(액화석유가스)' '경유' '휘발유 하이브리드' nan '기타연료' 'CNG(압축천연가스)' '전기'
 'LPG 하이브리드' '경유 하이브리드' '알코올' '등유' '수소' '태양열' 'LNG(액화천연가스)' 'CNG 하이브리드']

['A30S' 'A30L' 'A30M' 'A30C' nan]
['소형' '대형' '중형' '경형' nan]


# 전처리

## 등록&제원

In [75]:
cs_before = carr.merge(srcr, on='제원관리번호', how='left')
cs_before.shape

(31131025, 26)

## 등록&제원&법정동

In [76]:
cs_before['법정동코드'].isnull().sum()

0

In [77]:
cs_before['법정동코드_수정'] = cs_before['법정동코드'].astype('str')
cs_before['법정동코드_수정'].head()

0    2714012300
1    2823710800
2    4111312700
3    3014011000
4    4713012500
Name: 법정동코드_수정, dtype: object

In [78]:
cs_before['법정동코드_수정'] = cs_before['법정동코드_수정'].str[:5] + '00000'
cs_before['법정동코드_수정'] = pd.to_numeric(cs_before['법정동코드_수정'], errors='coerce')
cs_before[['법정동코드', '법정동코드_수정']].head()

,법정동코드,법정동코드_수정
0,2714012300,2714000000
1,2823710800,2823700000
2,4111312700,4111300000
3,3014011000,3014000000
4,4713012500,4713000000


In [79]:
csc_before = cs_before.merge(coder, on='법정동코드', how='left')
csc_before.shape

(31131025, 31)

In [80]:
csc_before.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '법정동코드_수정', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [81]:
csc_before[['시도', '시군구']].isnull().sum()

시도     23
시군구    49
dtype: int64

In [82]:
csc_before.loc[csc_before['시도'].isnull(), '법정동코드']

255191      5180031025
6001670     5180031033
7127782     5180031025
7852034     4165052000
8692073     5180031028
12526574    5180031028
12637600    5172035023
13739587    5175037022
13842700    5172035030
14915888    5180031023
15418353    5180031027
16291706    5180031031
16758757    5180031031
18306064    5180031023
18315819    5180031023
18402067    5180031023
18680038    5180031031
23468215    5172035031
30252963    4163055000
30721007    5172035024
30936586    5172035024
31086014    5180031023
31127915    5180031031
Name: 법정동코드, dtype: int64

In [83]:
csc_before_codey = csc_before[csc_before['시도'].notnull()]
csc_before_coden = csc_before[csc_before['시도'].isnull()]
csc_before_codey.shape[0] + csc_before_coden.shape[0] == csc_before.shape[0]

True

In [84]:
csc_before_coden.shape

(23, 31)

In [85]:
csc_before_coden = csc_before_coden.drop(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], axis=1)
csc_before_coden.shape

(23, 26)

In [86]:
csc_before_codenm = csc_before_coden.merge(coder, left_on='법정동코드_수정', right_on='법정동코드', how='left')
csc_before_codenm.shape

(23, 31)

In [87]:
csc_before_codenm.columns

Index(['차대번호', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종', '용도',
       '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료', '차종유형',
       '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량', '엔진출력',
       '법정동코드_수정', '법정동코드', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [88]:
csc_before_codenm[['시도', '시군구']].isnull().sum()

시도     0
시군구    0
dtype: int64

In [89]:
csc_before = pd.concat([csc_before_codey, csc_before_codenm], ignore_index=True)
csc_before.shape # 31131025

(31131025, 31)

In [90]:
csc_before[['시도', '시군구']].isnull().sum()

시도      0
시군구    26
dtype: int64

In [91]:
csc_before.loc[csc_before['시군구'].isnull(), ['시도', '시군구']]

,시도,시군구
747535,세종특별자치시,NaN
770310,세종특별자치시,NaN
1641011,경기도,NaN
1800621,세종특별자치시,NaN
5992843,세종특별자치시,NaN
6856490,세종특별자치시,NaN
6886850,세종특별자치시,NaN
8005829,세종특별자치시,NaN
8387500,세종특별자치시,NaN
8961169,세종특별자치시,NaN


### [출력] 등록&제원

In [92]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105'

In [138]:
# 2m 55.7s
csc_before.to_csv(os.path.join(pub_fold, f'등록_제원_법정동_전처리완료_{today_date_dtl}.csv'), index=False)

In [95]:
csc_before['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25895547
Y     5235478
Name: count, dtype: int64

In [118]:
len(csc_before['시도'].unique())

17

## LST

### 시도명 수정
- 강원도 -> 강원특별자치도
- 제주도 -> 제주특별자치도

In [115]:
print(cs['시도명'].unique())

array(['경상남도', '울산광역시', '전라남도', '서울특별시', '전라북도', '경기도', '경상북도', '강원특별자치도',
       '인천광역시', '부산광역시', '광주광역시', '제주특별자치도', '충청북도', '대전광역시', '충청남도',
       '대구광역시', '강원도', '세종특별자치시'], dtype=object)

In [143]:
cs.loc[cs['시도명'] == '강원도', '시도명'] = '강원특별자치도'
cs.loc[cs['시도명'] == '제주도', '시도명'] = '제주특별자치도'
print(cs['시도명'].unique())

array(['경상남도', '울산광역시', '전라남도', '서울특별시', '전라북도', '경기도', '경상북도', '강원특별자치도',
       '인천광역시', '부산광역시', '광주광역시', '제주특별자치도', '충청북도', '대전광역시', '충청남도',
       '대구광역시', '세종특별자치시'], dtype=object)

In [144]:
len(cs['시도명'].unique())

17

### [출력] LST

In [119]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105'

In [139]:
# 2m 55.3s
cs.to_csv(os.path.join(pub_fold, f'CEG_GRD_LST_전처리완료_{today_date_dtl}.csv'), index=False)

In [122]:
cs['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25918267
Y     5449290
Name: count, dtype: int64

# 분석

In [140]:
csc_before['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25895547
Y     5235478
Name: count, dtype: int64

In [141]:
cs['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25918267
Y     5449290
Name: count, dtype: int64

In [123]:
csc_before.shape, cs.shape

((31131025, 31), (31367557, 38))

## 차량 현황
- 말소여부, 등급, 지역, 연료별 현황

### 등록&제원

In [124]:
grp1_before = csc_before.groupby(['차량말소여부', '시도'], dropna=False)['차대번호'].count().unstack('시도').reset_index()
grp1_before.loc[grp1_before.shape[0]] = ['합계'] + grp1_before.iloc[:,1:].sum().to_list()
grp1_before['합계'] = grp1_before.iloc[:, 1:].sum(axis=1)
grp1_before # 31131025

시도,차량말소여부,강원특별자치도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도,제주특별자치도,충청남도,충청북도,합계
0,N,865371,6505925,1938726,1531987,721902,1256186,720579,1524100,3190988,197786,598554,1727278,1246241,991020,702169,1242387,934348,25895547
1,Y,20819,1498802,391443,361320,150139,261323,157404,260312,567439,38228,107286,332810,264304,235932,90021,288377,209519,5235478
2,합계,886190,8004727,2330169,1893307,872041,1517509,877983,1784412,3758427,236014,705840,2060088,1510545,1226952,792190,1530764,1143867,31131025


In [125]:
grp2_before = csc_before.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp2_before.columns = ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']
grp2_before.loc[grp2_before.shape[0]] = ['합계'] + grp2_before.iloc[:,1:].sum().to_list()
grp2_before['합계'] = grp2_before.iloc[:, 1:].sum(axis=1)
grp2_before # 31131025

,차량말소여부,1,2,3,4,5,등급_결측값,합계
0,N,3260843,10929631,8185490,1335156,923641,1260786,25895547
1,Y,171488,1323958,1739554,571754,1390709,38015,5235478
2,합계,3432331,12253589,9925044,1906910,2314350,1298801,31131025


In [126]:
grp2_before.columns

Index(['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값', '합계'], dtype='object')

In [127]:
grp3_before = csc_before.groupby(['차량말소여부', '시도', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp3_before.columns = ['차량말소여부', '시도', '1', '2', '3', '4', '5', '등급_결측값']
grp3_before.loc[grp3_before.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp3_before['1'].sum(), grp3_before['2'].sum(), grp3_before['3'].sum(), grp3_before['4'].sum(), grp3_before['5'].sum(), grp3_before['등급_결측값'].sum()]
grp3_before['합계'] = grp3_before.iloc[:, 2:].sum(axis=1)
grp3_before # 31131025

,차량말소여부,시도,1,2,3,4,5,등급_결측값,합계
0,N,강원특별자치도,93671.00,353126.00,295897.00,48886.00,36684.00,37107.00,865371.00
1,N,경기도,806161.00,2787982.00,2115473.00,313682.00,187428.00,295199.00,6505925.00
2,N,경상남도,216827.00,840372.00,600795.00,105397.00,82534.00,92801.00,1938726.00
3,N,경상북도,146821.00,623900.00,505781.00,105788.00,95392.00,54305.00,1531987.00
4,N,광주광역시,97038.00,314995.00,227003.00,34164.00,20757.00,27945.00,721902.00
5,N,대구광역시,156603.00,543240.00,395836.00,63778.00,41167.00,55562.00,1256186.00
6,N,대전광역시,96592.00,321374.00,216592.00,36185.00,20225.00,29611.00,720579.00
7,N,부산광역시,201128.00,644084.00,455874.00,68342.00,51589.00,103083.00,1524100.00
8,N,서울특별시,461096.00,1379431.00,945640.00,156575.00,106791.00,141455.00,3190988.00
9,N,세종특별자치시,29756.00,90447.00,58957.00,7113.00,3319.00,8194.00,197786.00


In [128]:
grp4_before = csc_before.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp4_before.columns = ['차량말소여부', '연료', '1', '2', '3', '4', '5', '등급_결측값']
grp4_before.loc[grp4_before.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp4_before['1'].sum(), grp4_before['2'].sum(), grp4_before['3'].sum(), grp4_before['4'].sum(), grp4_before['5'].sum(), grp4_before['등급_결측값'].sum()]
grp4_before['합계'] = grp4_before.iloc[:, 2:].sum(axis=1)
grp4_before # 31131025

,차량말소여부,연료,1,2,3,4,5,등급_결측값,합계
0,N,CNG 하이브리드,269.00,17.00,NaN,NaN,NaN,NaN,286.00
1,N,CNG(압축천연가스),NaN,22139.00,3905.00,1.00,7.00,2092.00,28144.00
2,N,LNG(액화천연가스),NaN,2.00,NaN,NaN,NaN,24.00,26.00
3,N,LPG 하이브리드,12559.00,NaN,NaN,NaN,NaN,2.00,12561.00
4,N,LPG(액화석유가스),447980.00,1117641.00,181145.00,23747.00,429.00,33318.00,1804260.00
5,N,경유,NaN,NaN,7298500.00,1004248.00,893733.00,343049.00,9539530.00
6,N,경유 하이브리드,NaN,NaN,18499.00,NaN,NaN,29660.00,48159.00
7,N,기타연료,NaN,NaN,5.00,34.00,20.00,73846.00,73905.00
8,N,등유,NaN,NaN,NaN,NaN,NaN,3.00,3.00
9,N,수소,33951.00,NaN,NaN,NaN,NaN,28.00,33979.00


#### 출력-통계 등록&제원

In [129]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105_PM_123217'

In [130]:
with pd.ExcelWriter(os.path.join(df_fold, f'기존등록제원_통계_{today_date_dtl}.xlsx')) as writer:
    grp1_before.to_excel(writer, sheet_name='말소_시도별', index=False)
    grp2_before.to_excel(writer, sheet_name='말소_등급별', index=False)
    grp3_before.to_excel(writer, sheet_name='말소_시도_등급별', index=False)
    grp4_before.to_excel(writer, sheet_name='말소_연료_등급별', index=False)

### LST

In [131]:
cs.shape

(31367557, 38)

In [132]:
grp1 = cs.groupby(['차량말소여부', '시도명'], dropna=False)['차대번호'].count().unstack('시도명').reset_index()
grp1.loc[grp1.shape[0]] = ['합계'] + grp1.iloc[:,1:].sum().to_list()
grp1['합계'] = grp1.iloc[:, 1:].sum(axis=1)
grp1 # 31498871

시도명,차량말소여부,강원특별자치도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도,제주특별자치도,충청남도,충청북도,합계
0,N,865992,6515691,1941447,1532871,722685,1256901,721575,1525701,3191576,197870,598936,1726651,1247142,991762,702619,1243436,935412,25918267
1,Y,199834,1509659,393882,363617,151047,262859,158342,261996,571182,38665,108047,335052,266001,237423,90658,290228,210798,5449290
2,합계,1065826,8025350,2335329,1896488,873732,1519760,879917,1787697,3762758,236535,706983,2061703,1513143,1229185,793277,1533664,1146210,31367557


In [133]:
grp2 = cs.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp2.columns = ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']
grp2.loc[grp2.shape[0]] = ['합계'] + grp2.iloc[:,1:].sum().to_list()
grp2['합계'] = grp2.iloc[:, 1:].sum(axis=1)
grp2 # 31498871

,차량말소여부,1,2,3,4,5,등급_결측값,합계
0,N,3277672,11661936,8550341,1328747,918555,181016,25918267
1,Y,176580,1378219,1802885,595265,1459801,36540,5449290
2,합계,3454252,13040155,10353226,1924012,2378356,217556,31367557


In [134]:
grp3 = cs.groupby(['차량말소여부', '시도명', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp3.columns = ['차량말소여부', '시도명', '1', '2', '3', '4', '5', '등급_결측값']
grp3.loc[grp3.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp3['1'].sum(), grp3['2'].sum(), grp3['3'].sum(), grp3['4'].sum(), grp3['5'].sum(), grp3['등급_결측값'].sum()]
grp3['합계'] = grp3.iloc[:, 2:].sum(axis=1)
grp3 # 31498871

,차량말소여부,시도명,1,2,3,4,5,등급_결측값,합계
0,N,강원특별자치도,94265.00,371046.00,310482.00,48699.00,36427.00,5073.00,865992.00
1,N,경기도,811461.00,2953598.00,2201009.00,312109.00,186513.00,51001.00,6515691.00
2,N,경상남도,218228.00,898896.00,626786.00,105101.00,82118.00,10318.00,1941447.00
3,N,경상북도,147528.00,648293.00,524259.00,105463.00,94733.00,12595.00,1532871.00
4,N,광주광역시,97599.00,330964.00,235064.00,34079.00,20656.00,4323.00,722685.00
5,N,대구광역시,157201.00,576442.00,412181.00,63499.00,40938.00,6640.00,1256901.00
6,N,대전광역시,97224.00,339416.00,224456.00,36100.00,20113.00,4266.00,721575.00
7,N,부산광역시,202333.00,705864.00,481207.00,68086.00,51244.00,16967.00,1525701.00
8,N,서울특별시,462851.00,1476702.00,978949.00,154837.00,106262.00,11975.00,3191576.00
9,N,세종특별자치시,29856.00,95197.00,60969.00,7088.00,3283.00,1477.00,197870.00


In [135]:
grp4 = cs.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp4.columns = ['차량말소여부', '연료', '1', '2', '3', '4', '5', '등급_결측값']
grp4.loc[grp4.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp4['1'].sum(), grp4['2'].sum(), grp4['3'].sum(), grp4['4'].sum(), grp4['5'].sum(), grp4['등급_결측값'].sum()]
grp4['합계'] = grp4.iloc[:, 2:].sum(axis=1)
grp4 # 31498871

,차량말소여부,연료,1,2,3,4,5,등급_결측값,합계
0,N,CNG 하이브리드,269.00,17.00,NaN,NaN,NaN,NaN,286.00
1,N,CNG(압축천연가스),NaN,24246.00,3837.00,1.00,7.00,NaN,28091.00
2,N,LNG(액화천연가스),NaN,26.00,NaN,NaN,NaN,NaN,26.00
3,N,LPG 하이브리드,12516.00,1.00,NaN,NaN,NaN,1.00,12518.00
4,N,LPG(액화석유가스),447362.00,1151212.00,180022.00,23697.00,429.00,83.00,1802805.00
5,N,경유,NaN,NaN,7639601.00,1000252.00,888727.00,4264.00,9532844.00
6,N,경유 하이브리드,NaN,NaN,48889.00,NaN,NaN,3.00,48892.00
7,N,기타연료,NaN,NaN,3.00,35.00,25.00,819.00,882.00
8,N,등유,NaN,NaN,NaN,NaN,NaN,2.00,2.00
9,N,수소,34059.00,NaN,NaN,NaN,NaN,NaN,34059.00


#### 출력-통계 LST

In [136]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105_PM_123234'

In [137]:
with pd.ExcelWriter(os.path.join(df_fold, f'RH제공파일_통계_{today_date_dtl}.xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='말소_시도별', index=False)
    grp2.to_excel(writer, sheet_name='말소_등급별', index=False)
    grp3.to_excel(writer, sheet_name='말소_시도_등급별', index=False)
    grp4.to_excel(writer, sheet_name='말소_연료_등급별', index=False)

# code end